In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [41]:
data_path = 'C:\\Users\\zer0nu11\\Documents\\grad\\skolkovohack2022\\data\\for_hack_2022\\'
test_path = 'C:\\Users\\zer0nu11\\Documents\\grad\\skolkovohack2022\\data\\for_hack_2022\\test\\'

In [42]:
data_jobs = pd.read_csv(data_path+'data_jobs.csv',sep=';')
data_candidates_workplaces = pd.read_csv(data_path+'data_candidates_work_places.csv',sep=';')
data_candidates_education = pd.read_csv(data_path+'data_candidates_education.csv',sep=';')
data_candidates = pd.read_csv(data_path+'data_candidates.csv',sep=';')

test_jobs = pd.read_csv(test_path+'test_jobs.csv',sep=';')
test_candidates_workplaces = pd.read_csv(test_path+'test_candidates_workplaces.csv',sep=';')
test_candidates_education = pd.read_csv(test_path+'test_candidates_education.csv',sep=';')
test_candidates = pd.read_csv(test_path+'test_candidates.csv',sep=';')

In [46]:
# assert test_jobs.shape[1] == data_jobs.shape[1] # test_jobs.shape БОЛЬШЕ НА 3 ФИЧИ!
# assert test_candidates.shape[1] == data_candidates.shape[1] # test_candidates.shape МЕНЬШЕ НА 4 ФИЧИ!
assert test_candidates_education.shape[1] == data_candidates_education.shape[1]
assert test_candidates_workplaces.shape[1] == data_candidates_workplaces.shape[1]